# 任务 5：调用 Bedrock 模型以生成代码

在此笔记本中，您将学习如何使用大型语言模型 (LLM) 来根据文本提示词生成代码。

此笔记本中使用的提示词是零样本提示词，因为除该提示词外，我们不提供任何文本示例。

为了演示 Amazon Bedrock 中模型的代码生成功能，您将了解代码生成的使用案例。您将探索使用 Boto3 客户端与 Amazon Bedrock API 进行通信，并向 API 提供由任务、指令和输入组成的输入，从而在不提供任何其他示例的情况下使后台模型生成输出。这样做的目的是演示强大的 LLM 如何轻松理解手头的任务并生成有说服力的输出。


## 场景

您是 AnyCompany 的数据分析师 Moe。该公司想了解过去一年不同产品的销售业绩。您拥有一个名为 sales.csv 的数据集。该数据集包含以下列：

- Date（格式为 YYYY-MM-DD）
- Product_ID（每个产品的唯一标识符）
- Price（每个产品的售价）


在此笔记本中，您将学习如何为给定提示词生成代码。您将使用基于 Amazon Bedrock API 与 Boto3 客户端的 Meta LLama 3。

## 任务 5.1：环境设置

在此任务中，您将设置环境。


In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## 任务 5.2：代码生成

在此任务中，您将为 Amazon Bedrock 服务准备输入内容，以便为您的使用案例生成 Python 程序。

#### 实验设置 - 为此实验创建示例 sales.csv 数据。



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## 任务 5.3：使用 Amazon Bedrock 生成的 Python 程序来分析销售情况

In [ ]:
# define prompt template
from langchain_core.prompts import PromptTemplate

def format_prompt(actor:str, input:str):
    match actor:
        case "user":
            prompt_template =  """<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""
            prompt = PromptTemplate.from_template(prompt_template)
            return prompt.format(actor=actor,input=input)
        case _:
            print("requested actor >" + actor + "< is not supported")
            return ""    

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- Total revenue by product 
- The product with the highest revenue 
- The date with the highest revenue and the revenue achieved on that date
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

"""
prompt=format_prompt("user",prompt_data)

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 2048,
    "temperature": 0,
    "top_p": 1,
    }) 
    

## 任务 5.4：调用模型

In [ ]:
modelId = "meta.llama3-8b-instruct-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
output_list = response_body.get("generation", [])
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"></i>（可选）从显示的输出中复制生成的代码，然后在下面单元中运行 Bedrock 生成的代码以进行验证。


现在，您已尝试使用了 `boto3` SDK，进而对 Amazon Bedrock API 有了基本了解。您使用此 API 生成了 Python 程序来分析给定销售数据并将其可视化。



### 自行尝试

- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件并继续执行**任务 6**。